## Imports

In [1]:
import bosdyn.client
import bosdyn.client.util
import bosdyn.client.robot_command as bdcrc
from bosdyn.client.robot_command import RobotCommandClient
from bosdyn.geometry import EulerZXY
from bosdyn.client.robot_command import RobotCommandBuilder
from bosdyn.client.image import ImageClient
from bosdyn.util import seconds_to_duration

from bosdyn.api import (arm_command_pb2, geometry_pb2, robot_command_pb2, synchronized_command_pb2,
                        trajectory_pb2)

import cv2
from groundlight import Groundlight
import pdb 
import time
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy

from PIL import Image
import io
np.random.seed(0)

## Setup

In [2]:
#Setup
sdk = bosdyn.client.create_standard_sdk('ASTRO')
robot = sdk.create_robot('192.168.50.3')
id_client = robot.ensure_client('robot-id')

#Authenticate

#works in python but not in notebook
#bosdyn.client.util.authenticate(robot)

#works everywhere
robot.authenticate('user', 'r4kr99yw3hje')

#State
robot_state_client = robot.ensure_client('robot-state')

#Create command client
command_client = robot.ensure_client(RobotCommandClient.default_service_name)

#image client
image_client = robot.ensure_client(ImageClient.default_service_name)

## Battery Checks

In [3]:
#Battery

def getBattInfo():
    
    robot_state = robot_state_client.get_robot_state()
    
    battery_state = robot_state.battery_states[0]
    
    chargePercent = battery_state.charge_percentage
    
    runtimeRemaining = battery_state.estimated_runtime
    
    return chargePercent.value, runtimeRemaining.seconds

In [71]:
chargePercent, runtimeRemaining = getBattInfo()

print("Battery at:", chargePercent, "%. Robot has", runtimeRemaining, "seconds left")

Battery at: 62.0 %. Robot has 3516 seconds left


## Lease

In [5]:
lease_client = robot.ensure_client('lease')
lease = lease_client.acquire()
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)

## Power

In [73]:
#Power on and time_sync

robot.power_on(timeout_sec=20)

print(robot.is_powered_on())

robot.time_sync.wait_for_sync()

True


## Movement

In [56]:
# SIT
cmd = RobotCommandBuilder.synchro_sit_command()

In [10]:
# STAND
cmd = RobotCommandBuilder.synchro_stand_command()

In [27]:
# SELFRIGHT
cmd = RobotCommandBuilder.selfright_command()

In [67]:
#Also requires end_time_secs info
frameTreeSnapshot = robot.get_frame_tree_snapshot()
cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, 0.1, frame_tree_snapshot = frameTreeSnapshot)

In [68]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

## Spin and take images

In [16]:
def moveToAndLookAt(moveTo, lookAt, blocking = True):
    """Takes in two 3 element arrays
        First is the coordinates for the gripper to move to
        Second is the coordinates for the gripper to look at
    """

    arm_pos_list = [moveTo[0],moveTo[1],moveTo[2]]
    dur_sec = 3

    duration = seconds_to_duration(dur_sec)


    arm_pos = geometry_pb2.Vec3(x=arm_pos_list[0], 
                                y=arm_pos_list[1], 
                                z=arm_pos_list[2])
    hand_pose = geometry_pb2.SE3Pose(position=arm_pos)
    hand_pose_traj_point = trajectory_pb2.SE3TrajectoryPoint(pose=hand_pose,
                                                             time_since_reference=duration)
    hand_traj = trajectory_pb2.SE3Trajectory(points=[hand_pose_traj_point])


    gaze_pos_list = [lookAt[0],lookAt[1],lookAt[2]]
    gaze_pos = geometry_pb2.Vec3(x=gaze_pos_list[0],
                                 y=gaze_pos_list[1],
                                 z=gaze_pos_list[2])
    gaze_pos_traj_point = trajectory_pb2.Vec3TrajectoryPoint(point=gaze_pos)
    gaze_traj = trajectory_pb2.Vec3Trajectory(points=[gaze_pos_traj_point])

    gaze_cmd = arm_command_pb2.GazeCommand.Request(target_trajectory_in_frame1=gaze_traj,
                                                   frame1_name='body',
                                                   tool_trajectory_in_frame2=hand_traj,
                                                   frame2_name='body')

    arm_command = arm_command_pb2.ArmCommand.Request(arm_gaze_command=gaze_cmd)
    synchronized_command = synchronized_command_pb2.SynchronizedCommand.Request(arm_command=arm_command)
    command = robot_command_pb2.RobotCommand(synchronized_command=synchronized_command)
    gaze_command_id = command_client.robot_command(command)
    if(blocking):
        bdcrc.block_until_arm_arrives(command_client, gaze_command_id, timeout_sec = time.time() + 3.0)

In [17]:
def blocking_stow():
    cmd = RobotCommandBuilder.arm_stow_command()
    cmd_id = command_client.robot_command(cmd)
    bdcrc.block_until_arm_arrives(command_client, cmd_id, timeout_sec = time.time() + 3.0)

In [18]:
def capAndShowImage(image_client, saveName):
    
    image_response = image_client.get_image_from_sources(["hand_color_image"])[0]
    image = Image.open(io.BytesIO(image_response.shot.image.data))
    image.save(saveName)

In [53]:
#Self right and stand up
bdcrc.blocking_selfright(command_client, timeout_sec = 10)
bdcrc.blocking_stand(command_client)

In [34]:
GROUNDLIGHT_API_TOKEN = 'api_2AaZ8QiCOvUFeHLJWGg8NTCPOAJ_uBTv8nu8U82Nv59ZHhi134HkAAirX4dwZw'

gl = Groundlight(api_token=GROUNDLIGHT_API_TOKEN)

# Call an API method (e.g., retrieve a list of detectors)
detectors = gl.list_detectors()

det = detectors.results[0]

In [74]:
#Self right and stand up
bdcrc.blocking_selfright(command_client, timeout_sec = 10)
bdcrc.blocking_stand(command_client)

cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

#Set the look at position
lookAt = [1.5, 0, -0.5]

for i in range(6):
    
    for i in range(5):
        #Move Arm to random position 
        xPosRand = np.random.uniform(0.6,0.9)
        yPosRand = np.random.uniform(-0.4,0.4)
        zPosRand = np.random.uniform(-0.1,0.8)

        moveTo = [xPosRand, yPosRand, zPosRand]
        moveToAndLookAt(moveTo, lookAt, blocking = True)

        print(moveTo)

        #CAP IMAGE AND CALL GL DETECTOR HERE
        capAndShowImage(image_client, 'handColorImage.jpg')
        image_query = gl.submit_image_query(detector_id=det.id, image = "handColorImage.jpg")
    
    blocking_stow()
    
    #Also requires end_time_secs info
    frameTreeSnapshot = robot.get_frame_tree_snapshot()
    rotate_cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, 2*np.pi/3, frame_tree_snapshot = frameTreeSnapshot)
    rotate_cmd_id = command_client.robot_command(rotate_cmd, end_time_secs = time.time()+5)
    bdcrc.block_for_trajectory_cmd(command_client, rotate_cmd_id)
    
    
bdcrc.blocking_sit(command_client)
    

[0.8824133114119497, 0.23936206988191333, 0.467403143180112]
[0.8622863899874842, -0.16558377239376265, 0.6640491997816265]
[0.7853630075752571, -0.38941051379288044, 0.21251016613899762]
[0.6444422582844495, 0.3854635118546026, 0.3305332763359893]
[0.7492174096495988, 0.11157801311897886, 0.2317261455166558]
[0.6410700815056797, 0.25769418655539644, 0.07086312071248216]
[0.7533956947639369, -0.2205463768202086, -0.011939963955369365]
[0.8586574552265049, 0.37833559121850424, 0.7647511922567002]
[0.8719666497663536, 0.21923786615891105, 0.19983063682577776]
[0.624330416996399, -0.07420706286895412, 0.10901072795384847]
[0.6397462904273948, -0.35725825457053983, 0.5530349277895209]
[0.6034282375875093, 0.21646459880222102, 0.03225198086033754]
[0.6238566247760267, -0.3283175726091157, 0.5048430266185231]
[0.6736101629558534, -0.06356842665592122, 0.4016319121915253]
[0.8581653521486381, 0.1816354101690626, 0.14329511471484319]
[0.6394448397873382, -0.35570054366304166, 0.171438771032848

In [50]:
bdcrc.blocking_sit(command_client)